In [2]:
import os
import stat
import subprocess
import numpy as np
import pandas as pd
from datetime import datetime

In [8]:
# the name of the directories containing all the normalized datasets
norm_dir = 'dchip2_mass_norm'
norm_data_dir = '15D_alphafactor_dun1_GSE49650_15D_clb1-6_GSE96997_alphafactor_dun1_GSE32974_GSE8799_cdc20_15D_GSE75694_dchip_20201123_092934'

### Make bash script for annotating all datasets in the norm_dir

In [9]:
# Make output directories for the mass normalizations if they don't exists already
datetime_stamp = datetime.today().strftime('%Y%m%d%H%M%S')
with open(f'batch_probe_annotate_file_{datetime_stamp}.sh', 'w+') as batch_file:
    batch_file.write('#!/bin/sh\n\n')
    batch_file.write('cd transcriptomics_process/annotate\n\n')
#     print('Commands added to bash file:')
    for file in os.listdir(os.path.join('../normalizations', norm_dir, norm_data_dir)):
        if file.endswith('.tsv') and 'readme' not in file and '_annot' not in file:
            data_path = os.path.join('../../../normalizations', norm_dir, norm_data_dir, file)
            data_col_id = 'probe'
            map_id = 'affy_yeast_2_35'
            map_col_id = 'probe'
            cmd = 'python attach_gene_info.py -data_path "{}" -data_col_id "{}" -map_id "{}" -map_col_id "{}"\n\n'.format(data_path, data_col_id, map_id, map_col_id)
#             print(cmd)
            batch_file.write(cmd)
    batch_file.write('cd -\n')
    
# Make bash file executable
st = os.stat(f'batch_probe_annotate_file_{datetime_stamp}.sh')
os.chmod(f'batch_probe_annotate_file_{datetime_stamp}.sh', st.st_mode | stat.S_IEXEC)
# os.chmod('batch_probe_annotate_file.sh', st.st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
# subprocess.check_call(['chmod', '+x', 'batch_probe_annotate_file.sh'])

In [10]:
# Run bash file and print output message
shellscript = subprocess.run([f'./batch_probe_annotate_file_{datetime_stamp}.sh'], stdin=subprocess.PIPE, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
# print(shellscript.stdout.decode('utf-8'))

### Make bash script for cleaning all datasets in the norm_dir
From the readme in the microarray_clean directory: </br>
Using an annotated tsv file, we clean out probes for affymetrix yeast s98 or 2.0 microarrays. Probes are removed if:
* they do not map to an SGD ID
* they do not map to a systematic name (e.g. YKR009C)
* the probe maps to multiple genes
* there are "questionable probes" (e.g. probes with _\<X>_at suffix, where X = s,f,g,x,i,r)

In [11]:
datetime_stamp = datetime.today().strftime('%Y%m%d%H%M%S')
with open(f'batch_clean_file_{datetime_stamp}.sh', 'w+') as batch_file:
    batch_file.write('#!/bin/sh\n\n')
    batch_file.write('cd transcriptomics_process/microarray_clean\n\n')
#     print('Commands added to bash file:')
    for file in os.listdir(os.path.join('../normalizations', norm_dir, norm_data_dir)):
        if file.endswith('_annot.tsv'):
            input_path = os.path.join('../../../normalizations', norm_dir, norm_data_dir, file)
            cmd = 'python clean_affy_yeast2.py -input_path "{}"\n\n'.format(input_path)
#             print(cmd)
            batch_file.write(cmd)
    batch_file.write('cd -\n')
    
# Make bash file executable
st = os.stat(f'batch_clean_file_{datetime_stamp}.sh')
os.chmod(f'batch_clean_file_{datetime_stamp}.sh', st.st_mode | stat.S_IEXEC)

In [12]:
# Run bash file and print output message
shellscript = subprocess.run([f'./batch_clean_file_{datetime_stamp}.sh'], stdin=subprocess.PIPE, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
# print(shellscript.stdout.decode('utf-8'))